In [26]:
!pip install nltk

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re

In [1018]:
import pandas as pd
df=pd.read_csv("다시 크롤링/Myeongdong Shopping Street.csv")
df["score"]=(df['STAR']>3.0)
df=df.replace(True,1)
df=df.replace(False,0)
print(df['score'].sum())
df

4955


,Unnamed: 0,STAR,CONTENT,score
0,0,5.0,Street food and shopping,1
1,1,4.0,always a symbol of Seoul though lots of shops ...,1
2,2,5.0,Convenient place for shopping,1
3,3,5.0,One of the famous streets of Korea If you wan...,1
4,4,5.0,It was crowded with people of various races,1
...,...,...,...,...
5725,5725,4.0,"Lots of shops and very busy, for the traveller...",1
5726,5726,5.0,The 9/F of Lotte Department Store is a duty fr...,1
5727,5727,4.0,"Myeongdong is a nice place for shopping, lot's...",1
5728,5728,5.0,I only had a few hours shopping at Myeongdong ...,1


In [1019]:
import re
def text_clearing(text):
    eng=re.compile('[^ a-z A-Z 0-9]+')
    #지정한 정규식에 해당하지 않은 것은 길이가 0인 문자열로 반환한다.
    result=eng.sub('',text)
    return result

In [1020]:
df["CONTENT"]=df["CONTENT"].apply(lambda x:text_clearing(x))
df

,Unnamed: 0,STAR,CONTENT,score
0,0,5.0,Street food and shopping,1
1,1,4.0,always a symbol of Seoul though lots of shops ...,1
2,2,5.0,Convenient place for shopping,1
3,3,5.0,One of the famous streets of Korea If you wan...,1
4,4,5.0,It was crowded with people of various races,1
...,...,...,...,...
5725,5725,4.0,Lots of shops and very busy for the traveller ...,1
5726,5726,5.0,The 9F of Lotte Department Store is a duty fre...,1
5727,5727,4.0,Myeongdong is a nice place for shopping lots o...,1
5728,5728,5.0,I only had a few hours shopping at Myeongdong ...,1


In [1021]:
import nltk
stopwords=nltk.corpus.stopwords.words('english')
def get_pos (x) :
    token=nltk.word_tokenize(x)
    pos = nltk.pos_tag(token)
    
    result = []
    
    for a1 in pos :
        if a1[0] not in stopwords:
            result.append(f'{a1[0]}/{a1[1]}')
    
    return result

In [1023]:
index_vectorizer=CountVectorizer(tokenizer=lambda x:get_pos(x))
X=index_vectorizer.fit_transform(df["CONTENT"].tolist())
X

<5730x16071 sparse matrix of type '<class 'numpy.int64'>'
	with 167919 stored elements in Compressed Sparse Row format>

In [1024]:
index_vectorizer.vocabulary_

{'street/NN': 13581,
 'food/NN': 5608,
 'shopping/NN': 12481,
 'always/RB': 860,
 'symbol/NN': 13933,
 'seoul/JJ': 12205,
 'though/IN': 14313,
 'lots/NNS': 8339,
 'shops/NNS': 12507,
 'closed/VBN': 2842,
 'myeongdong/NN': 9318,
 'still/RB': 13474,
 'fun/JJ': 5863,
 'place/NN': 10595,
 'convenient/JJ': 3231,
 'one/CD': 9933,
 'famous/JJ': 5231,
 'streets/NNS': 13606,
 'korea/NN': 7785,
 'want/VBP': 15391,
 'take/VB': 13981,
 'walk/NN': 15336,
 'evening/NN': 4914,
 'welcome/JJ': 15528,
 'crowded/VBN': 3544,
 'people/NNS': 10423,
 'various/JJ': 15129,
 'races/NNS': 11220,
 'great/JJ': 6234,
 'walk/VB': 15338,
 'night/NN': 9599,
 'check/VB': 2598,
 'vendors/NNS': 15169,
 'course/NN': 3402,
 'touristy/JJ': 14586,
 'cant/NN': 2321,
 'deny/IN': 3866,
 'good/JJ': 6149,
 'times/NNS': 14411,
 'family/NN': 5230,
 'highly/RB': 6691,
 'recommend/VBP': 11356,
 'chicken/NN': 2647,
 'kabob/NN': 7594,
 'many/JJ': 8618,
 'lively/RB': 8218,
 'old/JJ': 9908,
 'days/NNS': 3722,
 'lot/NN': 8328,
 'increased

In [1025]:
# 첫번째 행의 리뷰 이름
print(df["CONTENT"][0])
# 첫번째 행의 긍정/부정 시그널
print(df["score"][0])
# 첫번째 행의 index_vectorizer의 (형태소, 시리얼넘버) 
print(X[0])

Street food and shopping
1
  (0, 13581)	1
  (0, 5608)	1
  (0, 12481)	1


In [1026]:
y=df["score"]

In [1027]:
from sklearn.feature_extraction.text import TfidfTransformer
# TFidf 변환 모델 생성
tfidf_transformer = TfidfTransformer()
# 형태소 벡터 변환하기
X = tfidf_transformer.fit_transform(X)
for i in range(10):
    print(X[i])
    print("*"*20)


  (0, 13581)	0.5238580128766963
  (0, 12481)	0.6803371566489975
  (0, 5608)	0.5125564706719022
********************
  (0, 14313)	0.2759863237353902
  (0, 13933)	0.5451270735473874
  (0, 13474)	0.26962599103430895
  (0, 12507)	0.13695418473017237
  (0, 12205)	0.33157725369167157
  (0, 10595)	0.13119999906044652
  (0, 9318)	0.20448073150069299
  (0, 8339)	0.18212019431902124
  (0, 5863)	0.3192330238836409
  (0, 2842)	0.3966828963510443
  (0, 860)	0.27167868078296226
********************
  (0, 12481)	0.327522683563181
  (0, 10595)	0.3022863428754319
  (0, 3231)	0.8951825839808153
********************
  (0, 15528)	0.5968257562931988
  (0, 15391)	0.25707273783847023
  (0, 15336)	0.34682831102744377
  (0, 13981)	0.2959350841188193
  (0, 13606)	0.21991815059625452
  (0, 12481)	0.15040115736204207
  (0, 9933)	0.2070418833310643
  (0, 7785)	0.28290501495358006
  (0, 5231)	0.30144129556854815
  (0, 4914)	0.2917057216305493
********************
  (0, 15129)	0.43821261298038294
  (0, 11220)	0.7912

In [1028]:
print(X[0])
print(X.shape[0])

  (0, 13581)	0.5238580128766963
  (0, 12481)	0.6803371566489975
  (0, 5608)	0.5125564706719022
5730


In [1029]:
# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# LogisticRegression 
# penalty : 규제의 종류(l1, l2, elasticnet, none)
# C : 규제의 강도 
params = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model2 = LogisticRegression()
kfold = KFold(n_splits=10, shuffle=True, random_state = 1)
grid_clf2 = GridSearchCV(model2, param_grid=params, scoring='f1', cv=kfold)
grid_clf2.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf2.best_score_}')

최적의 하이퍼 파라미터 : {'C': 10, 'penalty': 'l2'}
최적의 모델 평균 성능 : 0.9342753761730025


In [1030]:
model = grid_clf2.best_estimator_

# 상관관계수 구하기
a1 = (model.coef_[0])
display(a1)
len(a1)

array([ 0.11704865,  0.81120903,  0.09234752, ...,  0.37099326,
        0.01485183, -0.41966373])

16071

In [1031]:
a2 = list(enumerate(a1))
a2

[(0, 0.11704864704002207),
 (1, 0.8112090278669791),
 (2, 0.09234751757967717),
 (3, 0.014070717735038051),
 (4, 0.03579669843973286),
 (5, 0.053852819557229),
 (6, 0.22982747637685746),
 (7, 0.07381688202817056),
 (8, 1.1906746286083445),
 (9, -0.6572766033918681),
 (10, 1.7217705714752551),
 (11, -1.1751156546438675),
 (12, -0.46351477335043034),
 (13, -0.6239017873111226),
 (14, -0.6053067873419266),
 (15, 0.02603633846380068),
 (16, -0.7474339196030868),
 (17, 0.05586974996893781),
 (18, 0.016551937067897756),
 (19, 0.024630159176421435),
 (20, 0.1604155624199684),
 (21, 0.09428221739602265),
 (22, 0.08539455076173036),
 (23, 0.1604155624199684),
 (24, 0.12809914761675004),
 (25, 0.25171976677540564),
 (26, -0.8044093201441987),
 (27, 0.06529635379216693),
 (28, 0.08948368780226314),
 (29, 0.17047183829176968),
 (30, 0.7762694092677942),
 (31, 0.510267613720831),
 (32, -1.4621012265640312),
 (33, 0.0985911044596807),
 (34, 0.15306879035560705),
 (35, 0.05733210965690869),
 (36, 0.6

In [1032]:
a3 = []

for idx, value in a2 : 
    a3.append((value, idx))
    
a3

[(0.11704864704002207, 0),
 (0.8112090278669791, 1),
 (0.09234751757967717, 2),
 (0.014070717735038051, 3),
 (0.03579669843973286, 4),
 (0.053852819557229, 5),
 (0.22982747637685746, 6),
 (0.07381688202817056, 7),
 (1.1906746286083445, 8),
 (-0.6572766033918681, 9),
 (1.7217705714752551, 10),
 (-1.1751156546438675, 11),
 (-0.46351477335043034, 12),
 (-0.6239017873111226, 13),
 (-0.6053067873419266, 14),
 (0.02603633846380068, 15),
 (-0.7474339196030868, 16),
 (0.05586974996893781, 17),
 (0.016551937067897756, 18),
 (0.024630159176421435, 19),
 (0.1604155624199684, 20),
 (0.09428221739602265, 21),
 (0.08539455076173036, 22),
 (0.1604155624199684, 23),
 (0.12809914761675004, 24),
 (0.25171976677540564, 25),
 (-0.8044093201441987, 26),
 (0.06529635379216693, 27),
 (0.08948368780226314, 28),
 (0.17047183829176968, 29),
 (0.7762694092677942, 30),
 (0.510267613720831, 31),
 (-1.4621012265640312, 32),
 (0.0985911044596807, 33),
 (0.15306879035560705, 34),
 (0.05733210965690869, 35),
 (0.69057

In [1033]:
coef_pos_index = sorted(a3, reverse=True)
coef_pos_index

[(5.827306956372757, 5608),
 (4.7305042525275836, 5187),
 (4.314795683794665, 684),
 (4.188643175575198, 3717),
 (4.175882406128334, 3818),
 (3.7209343297749116, 7706),
 (3.697089134180835, 1346),
 (3.6918266415166596, 9940),
 (3.433564158107149, 8376),
 (3.4078578017376997, 11602),
 (3.397052615185953, 3054),
 (3.3846713492191483, 865),
 (3.3415265911857976, 14776),
 (3.2653378592170945, 9004),
 (3.231286820071408, 2798),
 (3.2179508656202094, 12949),
 (3.1868634160510037, 1698),
 (3.1861070295799245, 538),
 (3.1742284977161552, 2398),
 (3.1595842464271677, 5245),
 (3.1193179455757214, 3835),
 (3.102988718660501, 6234),
 (3.0762845732722752, 774),
 (3.040160754549211, 5291),
 (2.981311793338124, 7929),
 (2.979615990042978, 7967),
 (2.9118107241008535, 8170),
 (2.90026659409744, 9256),
 (2.898230243722797, 13752),
 (2.8786336131044843, 584),
 (2.860614404524023, 12111),
 (2.76990652814398, 7986),
 (2.7661222857437933, 4030),
 (2.7563123485828385, 3324),
 (2.7554055582343224, 11472),
 (

In [1034]:
# 새로운 딕셔너리 생성
text_data_dict = {}

# 단어 사전에 있는 단어의 수만큼 반복한다.
for key in index_vectorizer.vocabulary_ :
	# 현재 key에 해당하는 값을 가져온다.
    value = index_vectorizer.vocabulary_[key]
    
    # 위의 딕셔너리에 담는다.
    text_data_dict[value] = key
    
text_data_dict

{13581: 'street/NN',
 5608: 'food/NN',
 12481: 'shopping/NN',
 860: 'always/RB',
 13933: 'symbol/NN',
 12205: 'seoul/JJ',
 14313: 'though/IN',
 8339: 'lots/NNS',
 12507: 'shops/NNS',
 2842: 'closed/VBN',
 9318: 'myeongdong/NN',
 13474: 'still/RB',
 5863: 'fun/JJ',
 10595: 'place/NN',
 3231: 'convenient/JJ',
 9933: 'one/CD',
 5231: 'famous/JJ',
 13606: 'streets/NNS',
 7785: 'korea/NN',
 15391: 'want/VBP',
 13981: 'take/VB',
 15336: 'walk/NN',
 4914: 'evening/NN',
 15528: 'welcome/JJ',
 3544: 'crowded/VBN',
 10423: 'people/NNS',
 15129: 'various/JJ',
 11220: 'races/NNS',
 6234: 'great/JJ',
 15338: 'walk/VB',
 9599: 'night/NN',
 2598: 'check/VB',
 15169: 'vendors/NNS',
 3402: 'course/NN',
 14586: 'touristy/JJ',
 2321: 'cant/NN',
 3866: 'deny/IN',
 6149: 'good/JJ',
 14411: 'times/NNS',
 5230: 'family/NN',
 6691: 'highly/RB',
 11356: 'recommend/VBP',
 2647: 'chicken/NN',
 7594: 'kabob/NN',
 8618: 'many/JJ',
 8218: 'lively/RB',
 9908: 'old/JJ',
 3722: 'days/NNS',
 8328: 'lot/NN',
 7164: 'inc

In [1035]:
# 긍정적인 어조 (상관계수가 1에 가장 큰)
top50 = coef_pos_index[:50]
# 부정적인 어조
bottom50 = coef_pos_index[-50:]

In [1036]:
import nltk
from nltk.corpus import stopwords

In [1037]:
nltk.download('english')
#불용어 추가
stop_words = set(stopwords.words('english'))
add_stop_words = ["'", ",", "'s", ":", ".", "?", "!","(",")","n't",
                 "korea","korean","good","like","awesome","perfect","nice","well","expensive","great","seoul","beautiful",
                 'wonderful','view','views','city','place','visit','tower','just',
                 'lotte','world','ride','rides',
                  'went','lotte','world','tower',
                  'street','lot','lots',
                  'market','ddp','insadong','places','shops','bookstore','bookstores','book','books','kyobo',
                  'lotte','aquarium','feel','animals','park','animals','coex','mall','shopping',
                  'amazing','6','1','40','8pm','1','40'
                 ]
for word in add_stop_words:
    stop_words.add(word)

[nltk_data] Error loading english: Package 'english' not found in
[nltk_data]     index


In [1038]:
pos=[]
for value, idx in top50 :
    if text_data_dict[idx].split('/')[0] not in stop_words:
        pos.append(text_data_dict[idx])
        print(text_data_dict[idx])

food/NN
facial/JJ
affordable/JJ
day/NN
definitely/RB
kinds/NNS
ones/NNS
love/VB
restaurants/NNS
compare/NN
try/VB
miss/VB
clean/JJ
socks/NNS
best/JJS
accessible/JJ
cash/NN
fantastic/JJ
delicious/JJ
alike/RB
favorite/JJ
ladies/NNS
language/NN
lining/VBG
must/MD
suggest/VBP
active/JJ
second/JJ
late/JJ
dinner/NN
cosmetics/NNS
remember/VB
loved/VBN
enter/VB
couple/NN
branded/VBD
atmosphere/RB
cheaper/JJR
hotel/NN
modern/JJ
closed/JJ
myeongdong/RB
streetfood/NN
safe/JJ
subway/NN
variety/NN


In [1039]:
neg=[]
for value, idx in bottom50 :
    if text_data_dict[idx].split('/')[0] not in stop_words:
        neg.append(text_data_dict[idx])
        print(text_data_dict[idx])

similar/JJ
repetition/NN
another/DT
areas/NNS
repeated/VBN
walkathlon/NN
however/RB
meal/NN
comfortably/RB
overpriced/VBN
personally/RB
check/NN
watching/VBG
shinjuku/NN
assorted/VBN
loud/JJ
flavour/NN
sells/VBZ
overrated/JJ
brandedfashion/NN
anymore/RB
follow/VB
pricey/JJ
riddled/VBD
better/JJR
honestly/RB
unless/IN
block/NN
disappointed/JJ
collection/NN
repetitive/JJ
average/JJ
elsewhere/RB
rather/RB
didnt/NN
poor/JJ
prize/NN
prefer/VBP
crowded/JJ
countries/NNS
bad/JJ
reallllllly/RB
less/RBR
mostly/RB
else/RB
touristy/JJ
rude/JJ
nothing/NN


In [1040]:
num_pos_rev=[]        
for i in pos:
    num_pos_rev.append(sum(df['CONTENT'].str.contains(i.split('/')[0])))

In [1041]:
num_neg_rev=[]        
for i in neg:
    num_neg_rev.append(sum(df['CONTENT'].str.contains(i.split('/')[0])))

In [1042]:
revword=pd.DataFrame([pos,num_pos_rev,neg,num_neg_rev],index=['긍정','리뷰 수','부정', '리뷰 수']).transpose()
revword

,긍정,리뷰 수,부정,리뷰 수
0,food/NN,3379,similar/JJ,74
1,facial/JJ,154,repetition/NN,7
2,affordable/JJ,87,another/DT,108
3,day/NN,855,areas/NNS,104
4,definitely/RB,242,repeated/VBN,31
5,kinds/NNS,112,walkathlon/NN,1
6,ones/NNS,141,however/RB,63
7,love/VB,674,meal/NN,69
8,restaurants/NNS,933,comfortably/RB,10
9,compare/NN,163,overpriced/VBN,38


In [1043]:
revword.to_csv("Myeongdong Shopping Street words.csv")

In [1044]:
keyword=input("키워드 입력 : ")
df_a = df[df['CONTENT'].str.contains(keyword)]
for i in df_a['CONTENT']:
    print("="*100)
    print(i)

키워드 입력 : overpriced
 I hated the Myeongdong commercial area because of the overpriced prices but now it has become a place of memories that I cant see even if I want to see it 
Items may be overpriced at street stalls May help if you do some research on what you want to buy Can find bargains too Huge variety of food clothes and shoes Avoid if you dislike rubbing shoulders with hoards of tourists its almost inevitable Appears to be very popular with Chinese tourists
This is a reasonably fine place for some overpriced street food and shopping  Please avoid the touts on the streets who will try to entice you with some cheap massage packages As the saying goes if it is to be true then it probably is Once you follow them to the massage parlor the sales staff there will pressurize you to take up very expensive sessions if you are not keen they would exert more high selling tactics Once you refused to take up any of these exorbitantly priced session and opt for the lower price one they would 